# PCA on Covariates Simulations Notebook

In [10]:
# Directory structure
import os
repo_dir = os.path.join(os.path.abspath(''), '../..')
input_dir = repo_dir + "/Input"
output_dir = repo_dir + "/Output"
figures_dir = output_dir + "/Figures"
tables_dir = output_dir + "/Tables"
sim_results_dir = output_dir + "/Sim_Results"

# Packages
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FactorAnalysis
import math
from tqdm import tqdm
from statsmodels.sandbox.regression.gmm import IV2SLS 

In [11]:
# Seed for reproducibility
np.random.seed(42)

In [12]:
# Supressing Output
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [13]:
# Data frame to store output
output = pd.DataFrame()

# 3,000 observations
N = 3000

# Loop over combinations of: betas, covariances between variables, p numbers of parameters
for beta1 in [0.1,1,10]:
    for beta2 in [0.1,1,10]:
        for covariance in [-0.9,-0.5,0,0.5,0.9]:
            for p in [5,20,50]:
                # Select only the scenarios we actually want to run by default - betas of 1, covariances of 0.5, p of 5... require three of these conditions to be satisfied to run.
                counter = 0
                if beta1 == 1:
                    counter += 1
                if beta2 == 1:
                    counter += 1
                if covariance == 0.5:
                    counter += 1
                if p == 5:
                    counter+=1
                if counter >= 3:
                    # Run with and without transformations
                    for me_cov in [0,0.5]:
                        # 1000 simulations
                        for k in tqdm(range(1000)):
                            # Initialize Lists to store coef values for all five methods and the true coef
                            pca_coef = []
                            mismeasured_coef = []
                            mismeasured_allvar_coef = []
                            mismeasured_avg_coef = []
                            iv_coef = []
                            true_val_coef =[]

                            # Create variables
                            vars_mean = [0,0,0]
                            vars_cov = np.array([[1,covariance,0],
                                                 [covariance,1,0],
                                                 [0,0,1]])
                            # Producing 3 variables: x for the variable of interest, the true Z covariate, the random error
                            vars_ = pd.DataFrame(np.random.multivariate_normal(vars_mean, vars_cov, N), columns = ['x','true_z','u'])
                            vars_['y'] = beta1 * vars_['x'] + beta2 * vars_['true_z'] + vars_['u']

                            # Create measurement errors for each of the p measurements of the covariates- mean zero and variance one
                            errors_mean = np.zeros(p)
                            errors_cov = np.zeros((p,p))
                            if me_cov != 0:
                                errors_cov[:] = me_cov
                            for i in range(p):
                                for j in range(p):
                                    if i == j:
                                        errors_cov[i,j] = 1
                                    

                            errors = np.random.multivariate_normal(errors_mean, errors_cov, N)
                            # Column labels for Z variables (covariates variables mismeasured)
                            z_vars = []
                            for i in range(p):
                                z_vars.append('z'+str(i+1))
                            # Add errors to the true_z to get mismeasured values
                            mismeasured_z = pd.DataFrame(errors, columns = z_vars)
                            for i in mismeasured_z.columns:
                                mismeasured_z[i] = mismeasured_z[i] + vars_['true_z']

                            # Do feature scaling (normalize to mean 0 and variance 1) for the mismeasured z
                            # Note that x and y are already normalized by construction
                            scaled_mismeasured_z = mismeasured_z.copy()
                            for i in mismeasured_z.columns:
                                scaled_mismeasured_z[i] = (mismeasured_z[i] - mismeasured_z[i].mean()) / mismeasured_z[i].std()

                            # Suppress output
                            with suppress_stdout():
                                # Use PCA on the mismeasured values
                                pca_model = PCA()
                                pca_results = pca_model.fit_transform(scaled_mismeasured_z)
                                pca_z = pca_results[:, 0]

                            # NOTE: in non-pca cases, no need to rescale or normalize since mismeasured variables and x and y have mean 0 and sd 1

                            # Average mismeasured variables:
                            vars_['avg_mismeasured_z'] = mismeasured_z[z_vars].mean(axis=1)

                            # Add relevant variables to vars_ dataframe
                            vars_[mismeasured_z.columns] = mismeasured_z
                            vars_['pca_z'] = pca_z

                            # Single mismeasured covariate results
                            X_mismeasured = vars_[['x','z1']]
                            X_mismeasured = sm.add_constant(X_mismeasured)
                            model_mismeasured = sm.OLS(vars_['y'],X_mismeasured)
                            results_mismeasured = model_mismeasured.fit()
                            mismeasured_coef.append(results_mismeasured.params[1])

                            # All Variables Mismeasured Results
                            # Create full list of items to include in regression
                            tot_vars = ['x']
                            tot_vars.extend(z_vars)
                            X_allvar = vars_[tot_vars]
                            X_allvar = sm.add_constant(X_allvar)
                            model_mismeasured_allvar = sm.OLS(vars_['y'],X_allvar)
                            results_mismeasured_allvar = model_mismeasured_allvar.fit()
                            mismeasured_allvar_coef.append(results_mismeasured_allvar.params[1])

                            # Average Mismeasured Variables Results
                            X_mismeasured_avg = vars_[['x','avg_mismeasured_z']]
                            X_mismeasured_avg = sm.add_constant(X_mismeasured_avg)
                            model_mismeasured_avg = sm.OLS(vars_['y'],X_mismeasured_avg)
                            results_mismeasured_avg = model_mismeasured_avg.fit()
                            mismeasured_avg_coef.append(results_mismeasured_avg.params[1])

                            # PCA Results
                            X_pca = vars_[['x','pca_z']]
                            X_pca = sm.add_constant(X_pca)
                            model_pca = sm.OLS(vars_['y'],X_pca)
                            results_pca = model_pca.fit()
                            pca_coef.append(results_pca.params[1])

                            # Instrumental Variables Results
                            # Instrument z1 on the other items in the mismeasured df
                            vars_ = sm.add_constant(vars_, has_constant='add')
                            iv_results = IV2SLS(endog = vars_['y'], exog = vars_[['const','x', 'z1']], instrument = pd.concat([vars_['x'], mismeasured_z.iloc[:, 1:]], axis = 1)).fit()
                            iv_coef.append(iv_results.params[1])

                            # True Results
                            X_true = vars_[['x','true_z']]
                            X_true = sm.add_constant(X_true)
                            model_true = sm.OLS(vars_['y'],X_true)
                            results_true = model_true.fit()
                            true_val_coef.append(results_true.params[1])

                            # Output Findings
                            new_output = pd.DataFrame()
                            new_output['mismeasured_coef'] = mismeasured_coef
                            new_output['mismeasured_allvar_coef'] = mismeasured_allvar_coef
                            new_output['mismeasured_avg_coef'] = mismeasured_avg_coef
                            new_output['pca_coef'] = pca_coef
                            new_output['true_val_coef'] = true_val_coef
                            new_output['iv_coef'] = iv_coef
                            new_output['covariance'] = vars_cov[0][1]
                            new_output['beta1'] = beta1
                            new_output['beta2'] = beta2
                            new_output['p'] = p
                            new_output['me_cov'] = me_cov
                            output = output.append(new_output)

output

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.03it/s]


,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,covariance,beta1,beta2,p,me_cov
0,0.431350,0.223271,0.223764,0.223645,0.112814,0.124550,0.5,0.1,1.0,5,0.0
0,0.387281,0.315756,0.314826,0.314799,0.109905,0.277551,0.5,0.1,1.0,5,0.5
0,1.001399,0.979398,0.979167,0.979228,0.971328,0.967482,0.5,1.0,0.1,5,0.0
0,1.028683,1.016913,1.016738,1.016729,0.969631,1.012168,0.5,1.0,0.1,5,0.5
0,0.256999,0.544957,0.543085,0.542795,1.032989,1.164268,-0.9,1.0,1.0,5,0.0
0,0.287684,0.351240,0.352574,0.352595,1.026005,0.363538,-0.9,1.0,1.0,5,0.5
0,0.708578,0.899876,0.899272,0.899321,0.999972,1.008806,-0.5,1.0,1.0,5,0.0
0,0.717243,0.774900,0.775291,0.775285,0.998116,0.773516,-0.5,1.0,1.0,5,0.5
0,0.978065,0.982528,0.983196,0.983333,0.983905,0.993426,0.0,1.0,1.0,5,0.0
0,0.996139,0.995827,0.995853,0.995821,0.997702,0.986970,0.0,1.0,1.0,5,0.5


In [26]:
output

,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,covariance,beta1,beta2,p,exp_of_var
0,0.343422,0.244492,0.442190,0.275556,0.095369,0.058960,0.5,0.1,1.0,5,yes
0,0.418633,0.311652,0.444835,0.341553,0.125920,0.155446,0.5,0.1,1.0,5,yes
0,0.359266,0.253938,0.395419,0.283112,0.101235,0.121762,0.5,0.1,1.0,5,yes
0,0.404182,0.274469,0.413487,0.303928,0.105303,0.133354,0.5,0.1,1.0,5,yes
0,0.381406,0.262020,0.399567,0.298677,0.083859,0.084940,0.5,0.1,1.0,5,yes
...,...,...,...,...,...,...,...,...,...,...,...
0,10.302065,10.113912,10.114272,10.114390,10.005734,10.014907,0.5,10.0,1.0,5,no
0,10.289174,10.098989,10.098718,10.098715,9.992232,9.984815,0.5,10.0,1.0,5,no
0,10.246537,10.074838,10.075841,10.075668,9.970239,9.956103,0.5,10.0,1.0,5,no
0,10.298014,10.109486,10.108630,10.108629,10.001683,9.984950,0.5,10.0,1.0,5,no


In [27]:
# Save raw coefficient results
output.to_csv(sim_results_dir + '/n_' + str(N) + '_results.csv')